## Agentes de Azure AI con Plugins Nativos y de Plantillas de Prompt

![Framework de Agentes del SDK de Semantic Kernel](https://github.com/kuljotSB/semantic-kernel/blob/main/Assets/Native_and_Prompt_Plugins.jpg?raw=true)

#### Instalación del SDK y las herramientas

In [ ]:
%pip install semantic-kernel==1.28.1, python-dotenv

#### Referenciando las bibliotecas y paquetes importantes

In [4]:
# Importaciones clave de Semantic Kernel y Python.
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings
from semantic_kernel.functions import KernelFunctionFromPrompt
from semantic_kernel.kernel import Kernel
from dotenv import load_dotenv
import os
import math
from typing import Annotated

from semantic_kernel.functions.kernel_function_decorator import kernel_function

#### Creando el Plugin Nativo de Matemáticas

In [5]:
# Esta clase define un conjunto de herramientas lógicas basadas en código Python puro.
class Math:
    """
    Descripción: MathPlugin proporciona un conjunto de funciones para realizar cálculos matemáticos.
    """

    # El decorador '@kernel_function' expone este método al Kernel como una habilidad.
    @kernel_function(
        description="Divide dos números.",
        name="Divide",
    )
    def divide(
        self,
        number1: Annotated[float, "el primer número para dividir"],
        number2: Annotated[float, "el segundo número por el cual dividir"],
    ) -> Annotated[float, "La salida es un número flotante"]:
        return float(number1) / float(number2)

    @kernel_function(
        description="Multiplica dos números. Al aumentar en un porcentaje, no olvides sumar 1 al porcentaje.",
        name="Multiply",
    )
    def multiply(
        self,
        number1: Annotated[float, "el primer número a multiplicar"],
        number2: Annotated[float, "el segundo número a multiplicar"],
    ) -> Annotated[float, "La salida es un número flotante"]:
        return float(number1) * float(number2)

    @kernel_function(
        description="Calcula la raíz cuadrada de un número",
        name="Sqrt",
    )
    def square_root(
        self,
        number1: Annotated[float, "el número del cual obtener la raíz cuadrada"],
    ) -> Annotated[float, "La salida es un número flotante"]:
        return math.sqrt(float(number1))

    @kernel_function(name="Add")
    def add(
        self,
        number1: Annotated[float, "el primer número a sumar"],
        number2: Annotated[float, "el segundo número a sumar"],
    ) -> Annotated[float, "la salida es un número flotante"]:
        return float(number1) + float(number2)

    @kernel_function(
        description="Resta un valor a otro valor",
        name="Subtract",
    )
    def subtract(
        self,
        number1: Annotated[float, "el primer número"],
        number2: Annotated[float, "el número a restar"],
    ) -> Annotated[float, "la salida es un número flotante"]:
        return float(number1) - float(number2)

#### Creando el Kernel con Comportamiento de Función Automático y los plugins nativos y de prompt

In [6]:
# Se cargan las variables de entorno.
load_dotenv()

# Se obtiene el nombre del modelo a utilizar.
model = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_MODEL")

# Se crea la instancia del Kernel.
kernel = Kernel()

service_id = "default"

# Se añade el servicio de IA de Azure al Kernel.
kernel.add_service(
    AzureChatCompletion(service_id=service_id,
                        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
                        deployment_name=os.getenv("AZURE_OPENAI_CHAT_COMPLETION_MODEL"),
                        endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )
)

# Se obtienen las configuraciones de ejecución del servicio de IA.
settings = kernel.get_prompt_execution_settings_from_service_id(service_id=service_id)

# Se configura el comportamiento de elección de función para invocar automáticamente las funciones del kernel.
# Esta es la línea que "enciende" la capacidad del agente para usar herramientas de forma autónoma.
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Se añade el plugin nativo de matemáticas al Kernel, dándole al agente la habilidad de calcular.
kernel.add_plugin(Math(), plugin_name="math")

# Se añade el plugin de plantilla de prompt al Kernel, dándole al agente la habilidad de formatear texto.
plugin = kernel.add_plugin(parent_directory = os.path.join(os.getcwd(), "plugins", "prompt_templates"), plugin_name = "basic_plugin")

#### Creando un Agente `ChatCompletionAgent` con el kernel

In [7]:
from semantic_kernel.functions import KernelArguments

# Se crea el agente.
agent = ChatCompletionAgent(
    kernel=kernel, # Se le pasa el Kernel, que ahora contiene el servicio de IA y los dos plugins.
    name="Plugins-Agent", # Nombre para el agente.
    instructions="Eres un asistente de IA muy útil", # Instrucciones base.
    arguments=KernelArguments(settings=settings), # Se aplican las configuraciones de ejecución automática de funciones.
)

#### Invocando al agente para ver la ejecución de los plugins

In [ ]:
# Este prompt es una solicitud de dos partes que requiere dos herramientas diferentes.
user_input = """suma 5 y 2. También crea la información de contacto para el usuario con los siguientes detalles:
1) Nombre: John Doe
2) Teléfono: 123-456-7890
3) Email: john@outlook.com
4) Dirección: 123 Main St, Springfield, USA
"""
# Se define la función asíncrona para llamar al agente.
async def get_response_from_agent():
    # El agente recibirá el input, y su configuración 'Auto' le permitirá
    # decidir llamar a la función 'math.Add' y luego a 'basic_plugin.contact_information'.
    response =  await agent.get_response(
        messages = user_input
    )
    
    return response

# Se ejecuta la llamada y se imprime la respuesta final, que combinará los resultados de ambas herramientas.
response = await get_response_from_agent()
print(response)

#### Viendo los pasos intermedios de ejecución del plugin con "intermediate_steps"

In [ ]:
# Importaciones para inspeccionar los contenidos de los mensajes internos del agente.
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.agents import ChatHistoryAgentThread
from semantic_kernel.contents import AuthorRole, ChatMessageContent, FunctionCallContent, FunctionResultContent

# Se define un hilo para la conversación.
thread = ChatHistoryAgentThread()

# Se define una lista vacía para almacenar los "pensamientos" o pasos intermedios del agente.
intermediate_steps: list[ChatMessageContent] = []

# Se define una función 'callback'. El agente llamará a esta función automáticamente
# cada vez que piense en llamar una herramienta o reciba un resultado de ella.
async def handle_intermediate_steps(message: ChatMessageContent) -> None:
    # Simplemente añade el mensaje intermedio a nuestra lista.
    intermediate_steps.append(message)

# Se define la función principal que invocará al agente y registrará los pasos.
async def get_response_from_agent_with_intermediate_steps():
    # Se usa 'agent.invoke' que permite usar callbacks, en lugar de 'get_response'.
    async for response in agent.invoke(
        messages=user_input, # La misma consulta del usuario.
        thread=thread, # El hilo de la conversación.
        on_intermediate_message=handle_intermediate_steps # Aquí "conectamos" nuestra función callback.
    ):
        # El bucle 'async for' se usa para manejar respuestas en streaming, aquí solo tomamos la primera respuesta completa.
        return response

# Se ejecuta la función.
response = await get_response_from_agent_with_intermediate_steps()
print("Respuesta del Agente: \n")
print(f"{response} \n")
print("------------------------------------------------------------")

 # Se imprimen los pasos intermedios que hemos capturado.
print("\nPasos Intermedios (Razonamiento del Agente):")
for msg in intermediate_steps:
     # Si el mensaje es el resultado de una función, lo imprimimos.
     if any(isinstance(item, FunctionResultContent) for item in msg.items):
         for fr in msg.items:
             if isinstance(fr, FunctionResultContent):
                 print(f"=> Resultado de Función: '{fr.result}' de la función: '{fr.name}'")
     # Si el mensaje es la decisión de llamar a una función, lo imprimimos.
     elif any(isinstance(item, FunctionCallContent) for item in msg.items):
         for fcc in msg.items:
             if isinstance(fcc, FunctionCallContent):
                 print(f"<= Llamada a Función: '{fcc.name}' con argumentos: {fcc.arguments}")
     # De lo contrario, es un mensaje normal.
     else:
         print(f"{msg.role}: {msg.content}")